In [2]:
import os
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
import tensorflow as tf

/Users/eurysee/Desktop/Pattern Recognition/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if mse == 0:
        return 100
    max_pixel = 1.0  # normalized images
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def load_images_from_folder(foldername, target_size=(64, 64)):
    images = []
    for filename in os.listdir(foldername):
        img = cv.imread(os.path.join(foldername, filename), cv.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv.resize(img, target_size)  
            images.append(img)
    images = np.array(images).astype('float32') / 255.
    images = np.expand_dims(images, axis=-1)
    return images

images = load_images_from_folder("pneumonia")
print("Loaded images shape:", images.shape)

Loaded images shape: (4273, 64, 64, 1)


In [11]:
def add_poisson_noise(images, lam):
    noisy = np.random.poisson(images * lam) / lam
    noisy = np.clip(noisy, 0., 1.)
    return noisy

lam_values = [25, 50, 75]
noisy_images = {lam: add_poisson_noise(images, lam) for lam in lam_values}

x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)
train_noisy = {}
test_noisy = {}
for lam in lam_values:
    train_noisy[lam], test_noisy[lam] = train_test_split(noisy_images[lam], test_size=0.2, random_state=42)

print("Data prepared and split.")

Data prepared and split.


In [12]:
def build_autoencoder():
    input_img = Input(shape=(64, 64, 1))  
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = Model(input_img, decoded)
    model.compile(optimizer='adam', loss='mse')
    return model

In [13]:
def build_autoencoder():
    input_img = Input(shape=(64, 64, 1))  
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = Model(input_img, decoded)
    model.compile(optimizer='adam', loss='mse')
    return model

In [14]:
results = {}
for lam in lam_values:
    print(f"\nTraining autoencoder for noise lambda={lam}...")
    model = build_autoencoder()
    model.fit(train_noisy[lam], x_train,
              epochs=5,
              batch_size=128,
              shuffle=True,
              validation_data=(test_noisy[lam], x_test),
              verbose=2)
    
    decoded_imgs = model.predict(test_noisy[lam])
    psnr_scores = [PSNR(x_test[i], decoded_imgs[i]) for i in range(len(x_test))]
    avg_psnr = np.mean(psnr_scores)
    print(f"Average PSNR for lambda={lam}: {avg_psnr:.2f} dB")
    results[lam] = avg_psnr



Training autoencoder for noise lambda=25...
Epoch 1/5
27/27 - 13s - 467ms/step - loss: 0.0251 - val_loss: 0.0100
Epoch 2/5
27/27 - 11s - 409ms/step - loss: 0.0074 - val_loss: 0.0061
Epoch 3/5
27/27 - 10s - 360ms/step - loss: 0.0056 - val_loss: 0.0050
Epoch 4/5
27/27 - 9s - 342ms/step - loss: 0.0048 - val_loss: 0.0048
Epoch 5/5
27/27 - 10s - 363ms/step - loss: 0.0046 - val_loss: 0.0044
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Average PSNR for lambda=25: 23.76 dB

Training autoencoder for noise lambda=50...
Epoch 1/5
27/27 - 10s - 374ms/step - loss: 0.0313 - val_loss: 0.0163
Epoch 2/5
27/27 - 9s - 324ms/step - loss: 0.0096 - val_loss: 0.0068
Epoch 3/5
27/27 - 11s - 407ms/step - loss: 0.0060 - val_loss: 0.0054
Epoch 4/5
27/27 - 9s - 347ms/step - loss: 0.0049 - val_loss: 0.0045
Epoch 5/5
27/27 - 9s - 323ms/step - loss: 0.0042 - val_loss: 0.0040
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Average PSNR for lambda=50: 24.30 dB

Training autoencoder for noise lambda=75...
Epoch 1/5
27/27 - 10s - 3

In [15]:
# Classical filters + evaluation helper
def evaluate_filter(test_noisy_images, test_clean_images, filter_func):
    psnr_vals = []
    for noisy_img, clean_img in zip(test_noisy_images, test_clean_images):
        denoised_img = filter_func(noisy_img)
        psnr_vals.append(PSNR(clean_img, denoised_img))
    return np.mean(psnr_vals)

def mean_blur(img):
    return cv.blur(img, (3, 3))

def median_blur(img):
    img_uint8 = (img * 255).astype(np.uint8)
    med = cv.medianBlur(img_uint8, 3)
    return med.astype(np.float32) / 255.

def bilateral_filter(img):
    img_uint8 = (img * 255).astype(np.uint8)
    bilateral = cv.bilateralFilter(img_uint8, 5, 75, 75)
    return bilateral.astype(np.float32) / 255.

def gaussian_blur(img):
    return cv.GaussianBlur(img, (3, 3), 0)

In [ ]:
# Evaluate filters
for lam in lam_values:
    print(f"\nEvaluating classical filters for lambda={lam}:")
    mean_psnr = evaluate_filter(test_noisy[lam], x_test, mean_blur)
    median_psnr = evaluate_filter(test_noisy[lam], x_test, median_blur)
    bilateral_psnr = evaluate_filter(test_noisy[lam], x_test, bilateral_filter)
    gaussian_psnr = evaluate_filter(test_noisy[lam], x_test, gaussian_blur)
    
    print(f"Mean Blur PSNR: {mean_psnr:.2f} dB")
    print(f"Median Blur PSNR: {median_psnr:.2f} dB")
    print(f"Bilateral Filter PSNR: {bilateral_psnr:.2f} dB")
    print(f"Gaussian Blur PSNR: {gaussian_psnr:.2f} dB")



Evaluating classical filters for lambda=25:
Mean Blur PSNR: 10.38 dB
Median Blur PSNR: 10.21 dB
Bilateral Filter PSNR: 10.33 dB
Gaussian Blur PSNR: 10.33 dB

Evaluating classical filters for lambda=50:
Mean Blur PSNR: 10.40 dB
Median Blur PSNR: 10.30 dB
Bilateral Filter PSNR: 10.40 dB
Gaussian Blur PSNR: 10.36 dB

Evaluating classical filters for lambda=75:
Mean Blur PSNR: 10.41 dB
Median Blur PSNR: 10.32 dB
Bilateral Filter PSNR: 10.42 dB
Gaussian Blur PSNR: 10.38 dB
